In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display
import requests
from bs4 import BeautifulSoup

import json


In [2]:
class LentaRu_parser:
    def __init__(self):
        pass


    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])
        print(url)

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])
        search_table['PostDate']=param_dict['dateFrom']

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 1,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')

            out = pd.concat([out, self._get_search_table(param_copy)], ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_csv("/tmp/checkpoint_table.csv")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_csv("lenta_{}_{}_new.csv".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out


<>:7: SyntaxWarning: invalid escape sequence '\ '
<>:7: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_4111/2594647819.py:7: SyntaxWarning: invalid escape sequence '\ '
  """


In [3]:
# Задаем тут параметры
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "4"
dateFrom = '2020-01-01'
dateTo = "2020-01-04"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2020-01-04', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '4', 'domain': '1'}


In [4]:
assert use_parser == "LentaRu"
parser = LentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                          time_step = 1,
                          save_every = 5,
                          save_excel = True)
print(len(tbl.index))
tbl.head()



https://lenta.ru/search/v2/process?from=0&size=1000&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=4&modified%2Cfrom=2020-01-01&modified%2Cto=2020-01-02&query=
https://lenta.ru/search/v2/process?from=0&size=1000&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=4&modified%2Cfrom=2020-01-03&modified%2Cto=2020-01-04&query=
Finish
37


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet,PostDate
0,943747,https://lenta.ru/news/2020/01/01/org/,В России утвердили стандарты органической еды,1577838649,1577838649,1,1,0,0,4,"[7, 8]",https://icdn.lenta.ru/images/2019/12/31/13/201...,1577838649,Фото: Rupert Oberhäuser / Globallookpress.com ...,Вступил в силу принятый в 2018 году закон,Фото: Rupert Oberhäuser / ... для производител...,2020-01-01
1,943634,https://lenta.ru/news/2020/01/01/theaters/,Российские театры освободили от налогов,1577840882,1577840883,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/30/20/201...,1577840882,Фото: Константин Кокошкин / «Коммерсантъ» Росс...,Обнулить базу удастся при условии строго следо...,Фото: Константин Кокошкин / «Коммерсантъ»... п...,2020-01-01
2,943811,https://lenta.ru/news/2020/01/01/tariff/,Покупать в интернете за границей стало сложнее,1577847420,1577958238,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/17/201...,1577847420,Фото: Depositphotos Покупать в интернете за гр...,Порог беспошлинного ввоза снижен до 200 евро з...,"Фото: Depositphotos Покупать в интернете ..., ...",2020-01-01
3,943816,https://lenta.ru/news/2020/01/01/gas/,На Украине подорожал газ,1577850424,1577850425,1,1,0,0,4,"[7, 8, 198]",https://icdn.lenta.ru/images/2019/12/31/18/201...,1577850424,Фото: Sean Gallup / Getty Images Цена на газ д...,Стоимость тысячи кубометров для населения дост...,"Фото: Sean Gallup / Getty Images Цена на ...,3...",2020-01-01
4,943729,https://lenta.ru/news/2020/01/01/self/,В России стало больше самозанятых,1577854128,1577854128,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/11/201...,1577854128,Фото: Олег Харсеев / «Коммерсантъ» В России ст...,Режим распространили на 19 новых регионов,Фото: Олег Харсеев / «Коммерсантъ» В ... распр...,2020-01-01


In [5]:
tbl

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet,PostDate
0,943747,https://lenta.ru/news/2020/01/01/org/,В России утвердили стандарты органической еды,1577838649,1577838649,1,1,0,0,4,"[7, 8]",https://icdn.lenta.ru/images/2019/12/31/13/201...,1577838649,Фото: Rupert Oberhäuser / Globallookpress.com ...,Вступил в силу принятый в 2018 году закон,Фото: Rupert Oberhäuser / ... для производител...,2020-01-01
1,943634,https://lenta.ru/news/2020/01/01/theaters/,Российские театры освободили от налогов,1577840882,1577840883,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/30/20/201...,1577840882,Фото: Константин Кокошкин / «Коммерсантъ» Росс...,Обнулить базу удастся при условии строго следо...,Фото: Константин Кокошкин / «Коммерсантъ»... п...,2020-01-01
2,943811,https://lenta.ru/news/2020/01/01/tariff/,Покупать в интернете за границей стало сложнее,1577847420,1577958238,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/17/201...,1577847420,Фото: Depositphotos Покупать в интернете за гр...,Порог беспошлинного ввоза снижен до 200 евро з...,"Фото: Depositphotos Покупать в интернете ..., ...",2020-01-01
3,943816,https://lenta.ru/news/2020/01/01/gas/,На Украине подорожал газ,1577850424,1577850425,1,1,0,0,4,"[7, 8, 198]",https://icdn.lenta.ru/images/2019/12/31/18/201...,1577850424,Фото: Sean Gallup / Getty Images Цена на газ д...,Стоимость тысячи кубометров для населения дост...,"Фото: Sean Gallup / Getty Images Цена на ...,3...",2020-01-01
4,943729,https://lenta.ru/news/2020/01/01/self/,В России стало больше самозанятых,1577854128,1577854128,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/11/201...,1577854128,Фото: Олег Харсеев / «Коммерсантъ» В России ст...,Режим распространили на 19 новых регионов,Фото: Олег Харсеев / «Коммерсантъ» В ... распр...,2020-01-01
5,943843,https://lenta.ru/news/2020/01/01/exxx/,Суд отменил штраф американской компании за нар...,1577854440,1577855850,1,1,0,0,4,[],https://icdn.lenta.ru/images/2020/01/01/07/202...,1577854440,Фото: Essam Al Sudani / Reuters Федеральный су...,Наложенный властями штраф признали неправомерным,Фото: Essam Al Sudani / Reuters ... за нарушен...,2020-01-01
6,943846,https://lenta.ru/news/2020/01/01/blgr/,Болгария начала получать газ из «Турецкого пот...,1577863471,1577863472,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/01/01/10/202...,1577863471,Фото: Сергей Гунеев / РИА Новости C 1 января Б...,Достигнутые договоренности позволят сэкономить...,Фото: Сергей Гунеев / РИА Новости C 1 ... газ ...,2020-01-01
7,943850,https://lenta.ru/news/2020/01/01/this_is_begin/,На Украине заявили о начале транспортировки га...,1577868017,1577868017,1,1,0,0,4,"[7, 52]",https://icdn.lenta.ru/images/2020/01/01/11/202...,1577868017,Фото: Facebook Украина начала поставки газа в ...,Первые кубометры топлива уже поступили в Европу,Фото: Facebook Украина начала поставки ... по ...,2020-01-01
8,943880,https://lenta.ru/news/2020/01/01/naftogaz/,«Нафтогаз» объявил о завершении отделения опер...,1577890757,1577890757,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2020/01/01/17/202...,1577890757,Фото: Стрингер / РИА Новости Отделение операто...,"По словам Андрея Коболева, «Нафтогаз» стал кли...",Фото: Стрингер / РИА Новости Отделение ... гру...,2020-01-01
9,942738,https://lenta.ru/articles/2020/01/02/ne_myaso/,На всех не хватит,1577912600,1577912600,2,1,0,0,4,"[9, 198]",https://icdn.lenta.ru/images/2019/12/30/13/201...,1577912600,Фото: Steffi Loos / Getty Images Дарья Волкова...,Человечеству угрожает глобальный голод. Мясу п...,"Фото: Steffi Loos / Getty Images Дарья ..., а ...",2020-01-01
